# Manual experiments with dynamics-related metrics

> ⚠️ This notebook should be run only on [Google Colab](https://colab.research.google.com/github/Dove6/WIMU10/blob/feature/dynamics-metrics/notebooks/colab_dynamics_metrics.ipynb)

In [ ]:
# @title Cloning the repository

!git clone -b feature/dynamics-metrics https://github.com/Dove6/WIMU10.git
!mv ./WIMU10/* ./
!rm -rf ./WIMU10

In [ ]:
# @title Installing required packages

%pip install -r ./requirements-colab.txt

In [10]:
# @title Importing tested methods

from wimu10 import (  # noqa: E402
    compute_dynamics_histogram,
    compute_dynamics_levels_histogram,
    compute_dynamics_variability,
    compute_dynamics_levels_variability,
    compute_dynamics_transition_matrix,
    compute_dynamics_levels_transition_matrix,
)

In [ ]:
# @title Setting up a dataset

from setup_dataset import download_muspy_midi  # noqa: E402

dataset = 'musicnet'  # @param ["musicnet", "maestro", "nesmdb", "emopia", "lakh"]

downloaded_dataset = download_muspy_midi(dataset)  # Performs required extraction and conversion

print(f'The chosen dataset ({type(downloaded_dataset).__name__}) has {len(downloaded_dataset)} elements.')

## Processing a file (of choice)

In [ ]:
from os import listdir  # noqa: E402
from pathlib import Path  # noqa: E402

from google.colab import files  # noqa: E402
import matplotlib.pyplot as plt  # noqa: E402
import muspy as mp  # noqa: E402
import numpy as np  # noqa: E402

from setup_dataset import DATA_RAW_PATH  # noqa: E402

# @markdown Choose a file from the dataset by putting its index here:

file_index = 329  # @param {type:"integer"}

# @markdown *The index should be the zero-based index of a file converted by MusPy when setting up the dataset.*

# @markdown ---

# @markdown Alternatively you can put a full (relative or absolute) path to a MIDI file to be processed here:

full_file_path = ''  # @param ["", "Upload your own MIDI file"] {allow-input: true}

# @markdown *Leave empty to use the file_index field. You can upload your own MIDI file by choosing "Upload your own MIDI file" from dropdown.*

full_path_used = full_file_path != ''

if full_path_used:
    if full_file_path == 'Upload your own MIDI file':
        uploaded_file = files.upload()
        if list(uploaded_file.keys()):
            full_file_path = list(uploaded_file.keys())[0]
        else:
            raise Exception('No file uploaded!')
    path = Path(full_file_path)
else:
    extension = '.json'
    first_dataset_entry = sorted(
        filter(
            lambda filename: filename.endswith(extension),
            listdir(Path(f'{DATA_RAW_PATH}{dataset}/_converted/'))
        )
    )[0]
    first_dataset_entry = first_dataset_entry[:-len(extension)]
    padding_len = len(first_dataset_entry)
    path = Path(f'{DATA_RAW_PATH}{dataset}/_converted/{str(file_index).zfill(padding_len)}{extension}')

if not path.exists() or not path.is_file():
    raise Exception(f'There is no file at path: {path}')

if full_path_used:
    music = mp.read_midi(path)
else:
    music = mp.load_json(path)

print(f'Loaded file: {path} (source filename: {music.metadata.source_filename}, title: {music.metadata.title})')

resolution = int(music.resolution / 4)  # MIDI resolution is in quarternotes, divided by 4 gives 1/16 note accuracy
track_end = music.get_end_time()
total_scores = []
dynamics_levels = ['ppp', 'pp', 'p', 'mp', 'mf', 'f', 'ff', 'fff']  # Default dynamics levels used

for track_idx, track in enumerate(music.tracks):
    print()
    print(f'=== PROCESSING TRACK #{track_idx} ===')
    if len(track.notes) == 0:
        total_scores.append(None)
        print(f'  Skipping track {track_idx} (it has 0 notes).')
        continue

    # Calculate all metrics
    bins = compute_dynamics_histogram(music, track_idx, resolution=resolution, track_end=track_end)
    level_bins = compute_dynamics_levels_histogram(music, track_idx, resolution=resolution, track_end=track_end)
    variability = compute_dynamics_variability(music, track_idx, resolution=resolution, track_end=track_end)
    level_variability = compute_dynamics_levels_variability(music, track_idx, resolution=resolution, track_end=track_end)
    transition_matrix = compute_dynamics_transition_matrix(music, track_idx, resolution=resolution, track_end=track_end)
    level_transition_matrix = compute_dynamics_levels_transition_matrix(
        music, track_idx, resolution=resolution, track_end=track_end
    )

    # Add metrics to a list (for saving it to a file later)
    total_scores.append(
        {
            'histogram': bins,
            'level_histogram': level_bins,
            'variability': variability,
            'level_variability': level_variability,
            'transition_matrix': transition_matrix.tolist(),
            'level_transition_matrix': level_transition_matrix.tolist(),
        }
    )

    # Plotting histograms
    _, (a1, a2) = plt.subplots(1, 2, figsize=(12, 4))
    a1.bar(range(len(bins)), bins)
    a1.set_title('Velocity values histogram')
    a2.bar(range(len(level_bins)), level_bins)
    a2.set_title('Dynamics levels histogram')
    a2.set_xticks(range(len(dynamics_levels)), dynamics_levels)
    plt.show()

    # Plotting transition matrices
    fig, (a1, a2) = plt.subplots(1, 2, figsize=(12, 6))
    plt.colorbar(a1.imshow(transition_matrix), ax=a1, fraction=0.03)
    a1.set_title('Velocity values transition matrix')
    plt.colorbar(a2.imshow(level_transition_matrix), ax=a2, fraction=0.03)
    a2.set_title('Dynamics levels transition matrix')
    a2.set_xticks(range(len(dynamics_levels)), dynamics_levels)
    a2.set_yticks(range(len(dynamics_levels)), dynamics_levels)
    plt.show()

    # Plotting transition matrices with zeroed diagonal
    np.fill_diagonal(transition_matrix, 0)
    np.fill_diagonal(level_transition_matrix, 0)
    fig, (a1, a2) = plt.subplots(1, 2, figsize=(12, 6))
    plt.colorbar(a1.imshow(transition_matrix), ax=a1, fraction=0.03)
    a1.set_title('Velocity values transition matrix (with zeroed diagonal)')
    plt.colorbar(a2.imshow(level_transition_matrix), ax=a2, fraction=0.03)
    a2.set_title('Dynamics levels transition matrix (with zeroed diagonal)')
    a2.set_xticks(range(len(dynamics_levels)), dynamics_levels)
    a2.set_yticks(range(len(dynamics_levels)), dynamics_levels)
    plt.show()

    # Describing variability
    print(f'Velocity values variability: {variability.ratio[0] * 100:.2f}% ({variability.changes_count[0]} / {variability.total_count})')
    print('Dynamics levels variability:')
    print(f'- lesser changes (1..2 levels): {level_variability.ratio[0] * 100:.2f}% ({level_variability.changes_count[0]} / {level_variability.total_count})')
    print(f'- greater changes (3+ levels): {level_variability.ratio[1] * 100:.2f}% ({level_variability.changes_count[1]} / {level_variability.total_count})')


In [49]:
# @title Saving results

from datetime import datetime  # noqa: E402
import json  # noqa: E402
from os import makedirs  # noqa: E402

results_dir = './results/'
makedirs(results_dir, exist_ok=True)
current_datetime = datetime.now().strftime('%Y%m%d-%H%M%S')
results_filename = f'{results_dir}{path.stem}_{current_datetime}.json'
with open(results_filename, 'w') as f:
    json.dump(total_scores, f, indent=4)
    print(f'Results saved to {results_filename}')